In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

pd.set_option('display.float_format', '{:.2f}'.format)

In [67]:
data = pd.read_csv('blackrock_2022_Q1_holdings_value_esgrisk.csv')
df = data.copy()

df

,Unnamed: 0,Stock,Company Name,Sector,Shares Held or Principal Amt,ESG Risk,Market Value,% of Portfolio
0,0,AAPL,Apple Inc.,INFORMATION TECHNOLOGY,"1,027,632,887",17.00,"$179,434,978,000",4.74
1,1,MSFT,Microsoft Corporation,INFORMATION TECHNOLOGY,"523,749,870",15.00,"$161,477,322,000",4.26
2,2,AMZN,Amazon.com Inc.,CONSUMER DISCRETIONARY,"582,877,640",30.00,"$95,007,597,000",2.51
3,3,BP,BP p.l.c.,ENERGY,"1,950,622,815",NaN,"$63,141,660,522",1.67
4,4,TSLA,Tesla Inc.,CONSUMER DISCRETIONARY,"55,241,453",29.00,"$59,528,191,000",1.57
...,...,...,...,...,...,...,...,...
6054,6054,NLTX,NaN,NaN,"910,842",NaN,NaN,NaN
6055,6055,PRIVATE,NaN,NaN,NaN,NaN,NaN,NaN
6056,6056,HYGI,NaN,NaN,"100,010",NaN,NaN,NaN
6057,6057,AGRH,NaN,NaN,"100,010",NaN,NaN,NaN


#### Data Observation, Cleaning

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6059 entries, 0 to 6058
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    6059 non-null   int64  
 1   Stock                         6059 non-null   object 
 2   Company Name                  3965 non-null   object 
 3   Sector                        5736 non-null   object 
 4   Shares Held or Principal Amt  5885 non-null   object 
 5   ESG Risk                      456 non-null    float64
 6   Market Value                  5839 non-null   object 
 7   % of Portfolio                5839 non-null   float64
dtypes: float64(2), int64(1), object(5)
memory usage: 378.8+ KB


In [69]:
df['Market Value'] = df['Market Value'].str.replace('[\$,]', '', regex = True).astype(float)

In [70]:
df = df.rename(columns = {'% of Portfolio': 'Portfolio Percentage'})

In [71]:
df.dropna(axis=0, inplace=True)

In [72]:
df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [73]:
df.duplicated().sum()

np.int64(18)

In [74]:
df.describe()

,ESG Risk,Market Value,Portfolio Percentage
count,445.00,445.00,445.00
mean,21.70,6303450776.70,0.17
std,7.18,15892038737.53,0.42
min,7.00,483242000.00,0.01
25%,16.00,1652392000.00,0.04
50%,21.00,2804987000.00,0.07
75%,27.00,5437392000.00,0.14
max,46.00,179434978000.00,4.74


In [75]:
df.head()

,Stock,Company Name,Sector,Shares Held or Principal Amt,ESG Risk,Market Value,Portfolio Percentage
0,AAPL,Apple Inc.,INFORMATION TECHNOLOGY,"1,027,632,887",17.00,179434978000.00,4.74
1,MSFT,Microsoft Corporation,INFORMATION TECHNOLOGY,"523,749,870",15.00,161477322000.00,4.26
2,AMZN,Amazon.com Inc.,CONSUMER DISCRETIONARY,"582,877,640",30.00,95007597000.00,2.51
4,TSLA,Tesla Inc.,CONSUMER DISCRETIONARY,"55,241,453",29.00,59528191000.00,1.57
5,GOOGL,Alphabet Inc.,COMMUNICATIONS,"20,702,430",24.00,57580702000.00,1.52


In [76]:
df['Sector'].unique()

array(['INFORMATION TECHNOLOGY', 'CONSUMER DISCRETIONARY',
       'COMMUNICATIONS', 'HEALTH CARE', 'FINANCE', 'CONSUMER STAPLES',
       'ENERGY', 'UTILITIES AND TELECOMMUNICATIONS', 'TRANSPORTS',
       'REAL ESTATE', 'INDUSTRIALS', 'MATERIALS'], dtype=object)

#### EDA

In [77]:
sector = df.groupby('Sector')['Market Value'].sum()
top_sector = sector.sort_values(ascending = False).head(5).reset_index()

fig = px.pie(top_sector,
                names = 'Sector',
                values = 'Market Value',
                title = 'BlackRock Market Distribution'
)

fig.update_layout(
    width = 1000,
    height = 600
)

fig.show()

In [78]:
companies = df.groupby('Company Name')['Portfolio Percentage'].sum()
top_companies = companies.sort_values(ascending = False).head(10).reset_index()


fig = px.bar(top_companies,
             x = 'Company Name',
             y = 'Portfolio Percentage',
             title = 'BlackRock Stock Percentage',
)

fig.update_traces(marker = dict(color = 'lightgreen'))

fig.update_layout(width = 1200, height = 800,)

fig.show()

In [79]:
real_estate = df[df['Sector'] == 'REAL ESTATE']

real_estate_sort = real_estate.sort_values(by = 'Market Value', ascending= False).head(5)

fig = px.bar(real_estate_sort,
             x='Company Name',
             y='Market Value',
             title='Top 5 Real Estate Stock Holdings',
             labels={'Market Value': 'Market Value (USD)'},
             color='Stock'
             )

fig.update_layout(width=1000, height=600)

fig.show()

In [80]:
consumer = df[df['Sector'] == 'CONSUMER DISCRETIONARY']

consumer_sort = consumer.sort_values(by = 'Market Value', ascending= False).head(5)

fig = px.bar(consumer_sort,
             x='Company Name',
             y='Market Value',
             title='Top 5 Consumer Company Holdings',
             labels={'Market Value': 'Market Value (USD)'},
             color='Stock',  
             )

fig.update_layout(width=1000, height=600)

fig.show()

In [81]:
def top_company_sector(df):
    sectors = df['Sector'].unique()

    top_companies = []

    for sector in sectors:
        sector_df = df[df['Sector'] == sector]
        
        top_company = sector_df.loc[sector_df['Market Value'].idxmax()]
        
        top_companies.append(top_company)

    top_companies_df = pd.DataFrame(top_companies)
    
    top_companies_df = top_companies_df.sort_values(by='Market Value', ascending=False)

    fig = px.bar(top_companies_df,
                 x='Company Name',
                 y='Market Value',
                 color='Sector',
                 title='Top Company by Market Value in Each Sector',
                 labels={'Market Value': 'Market Value (USD)'})
    
    fig.update_layout(width=1400, height=850)
    fig.show()

top_company_sector(df)